# 分析不合规合同


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path


In [2]:
pd.options.display.max_rows = 5


In [3]:
data_dir_name = 'data'
data_dir = Path.cwd() / data_dir_name
if not data_dir.exists():
    data_dir.mkdir()
data_dir


PosixPath('/Users/levin/workspace/git-repositories/anaconda/study-pandas-tutorials/Work/data')

In [7]:
# lp: last period
lp_filename = 'non-compliant_contracts-2022-01-26.xlsx'
lp_sheet_name = 'Sheet1'

# tp: this period
tp_filename = 'non-compliant_contracts-2022-02-18.xlsx'
tp_sheet_name = 'Sheet1'

# common
key_fields = ['合同编号']


## 读取上期数据


In [8]:
lp_path = data_dir / lp_filename
lp_path


PosixPath('/Users/levin/workspace/git-repositories/anaconda/study-pandas-tutorials/Work/data/non-compliant_contracts-2022-01-26.xlsx')

In [9]:
lp_db = pd.read_excel(lp_path, lp_sheet_name)
# deduplication based on 'contract number' field
lp_db = lp_db.drop_duplicates(subset=key_fields)
lp_db


,序号,分公司,项目部,organ_id,项目名称,资源名称,合同编号,甲方名称,乙方名称,情况,说明
0,1,产园-杭州公司,上海森兰美奂创库,1427213,森兰美奂创库,"森兰美奂大厦A栋B座6层-641-CK671,森兰美奂大厦A栋B座6层-641-CK672,...",slmhck-2022-01-1242,上海浦隽房地产开发有限公司,上海云至国际货运代理有限公司,倒签,已审批
1,2,产园-杭州公司,上海森兰美奂创库,1427213,森兰美奂创库,"森兰美奂大厦A栋B座6层-627-CK553,森兰美奂大厦A栋B座6层-627-CK555,...",slmhck-2022-01-1240,上海浦隽房地产开发有限公司,上海以鼎科技发展有限责任公司,倒签,已审批
...,...,...,...,...,...,...,...,...,...,...,...
155,156,园区运营中心,园区运营中心其他,1413268,新时代北商业用地,"新时代北商业用地1层-游乐场,新时代北商业用地1层-足球场",xsdbsyyd-2022-01-1001,招商局蛇口工业区控股股份有限公司,深圳市鸿荣业投资有限公司,倒签,已审批
156,157,园区运营中心,园区运营中心其他,1413268,新时代北商业用地,"新时代北商业用地1层-游乐场,新时代北商业用地1层-足球场",xsdbsyyd-2021-10-1000,招商局蛇口工业区控股股份有限公司,深圳市鸿荣业投资有限公司,应结未结,已终止未结算


## 读取本期数据


In [10]:
tp_path = data_dir / tp_filename
tp_path


PosixPath('/Users/levin/workspace/git-repositories/anaconda/study-pandas-tutorials/Work/data/non-compliant_contracts-2022-02-18.xlsx')

In [11]:
tp_db = pd.read_excel(tp_path, tp_sheet_name)
tp_db = tp_db.drop_duplicates(subset=key_fields)
tp_db


,序号,分公司,项目部,organ_id,项目名称,资源名称,合同编号,甲方名称,乙方名称,情况,说明
0,1,产园-深圳公司,蛇口网谷,1032,科健大厦,科健大厦-广告位2,kjds-2022-02-0109,深圳市招商创业有限公司,驰众广告有限公司,倒签,已审批
1,2,产园-深圳公司,零星物业,1020,招港大厦,"招港大厦7层-704,招港大厦7层-705",zgds-2022-01-0056,深圳市招商创业有限公司,深圳市大田粮食有限公司,倒签,已审批
...,...,...,...,...,...,...,...,...,...,...,...
255,256,产园-深圳公司,蛇口网谷,1431205,万维大厦,万维大厦2层-203,wwds-2021-06-1028,深圳市招商创业有限公司,迈威迩电子科技（北京）有限公司深圳分公司,应结未结,已终止未结算
256,257,产园-深圳公司,蛇口网谷,1430201,万海大厦,万海大厦B座5层-501A,whds-2021-09-1094,深圳市万海大厦管理有限公司,胡艳芬,应结未结,已终止未结算


## 提取增量数据

从本期数据中提取本期增量数据, 为了分析本期增量情况.

使用 `pandas` 做这种操作稍微有些复杂, 基本思路如下:

1. 使用 `left join` 模式将本期数据与上期数据进行 `merge`, 本期数据作为 `left`
2. 在使用 `merge` 函数时, 需要设置 `indicator` 参数为 `True`. 这样就可以在 `merge` 的结果表中增加一列用于表明 `merge` 的方式
3. 利用 `indicator` 来创建一个 Boolean indexing, 从而可以提取出增量数据


In [12]:
db_all = tp_db.merge(
    lp_db,
    on=key_fields,
    how='left',
    indicator=True,
    suffixes=(None, '_y')
)
incremental_flag = db_all['_merge'] == 'left_only'
incremental_db = db_all[tp_db.columns][incremental_flag]
incremental_db


,序号,分公司,项目部,organ_id,项目名称,资源名称,合同编号,甲方名称,乙方名称,情况,说明
0,1,产园-深圳公司,蛇口网谷,1032,科健大厦,科健大厦-广告位2,kjds-2022-02-0109,深圳市招商创业有限公司,驰众广告有限公司,倒签,已审批
2,3,产园-重庆公司,金山意库,1412212,金山意库,1号楼-灯箱广告位001号,jsyk-2022-01-0656,重庆招商金山意库商业管理有限公司,众凯重庆智能科技有限公司,倒签,已审批
...,...,...,...,...,...,...,...,...,...,...,...
197,256,产园-深圳公司,蛇口网谷,1431205,万维大厦,万维大厦2层-203,wwds-2021-06-1028,深圳市招商创业有限公司,迈威迩电子科技（北京）有限公司深圳分公司,应结未结,已终止未结算
198,257,产园-深圳公司,蛇口网谷,1430201,万海大厦,万海大厦B座5层-501A,whds-2021-09-1094,深圳市万海大厦管理有限公司,胡艳芬,应结未结,已终止未结算


## 分析本期数据

分析思路:

1. 根据 `['分公司', '项目部', '情况']` 分组, 计算数量
2. 分组小计
3. 按分组小计倒排序


In [13]:
def analyze(_df):
    tp_grp = _df.groupby(['分公司', '项目部', '情况'])
    tp_grp_unstacked = tp_grp['organ_id'].count().unstack([-1, -2])
    subtotal = '--小计--'
    tp_grp_unstacked[('', subtotal)] = \
        tp_grp_unstacked.apply(lambda s: s.sum(), axis=1)

    tp_grp_stacked = tp_grp_unstacked.sort_values(
        ('', subtotal), ascending=False).stack([-1, -2]).to_frame('数量')
    tp_grp_stacked.loc[('总计', '', '')] = tp_grp['organ_id'].count().sum()

    return tp_grp_stacked.astype('int32')


In [14]:
analyze(tp_db)


数量
分公司     项目部      情况       
产园-杭州公司 --小计--          69
        上海森兰美奂创库 倒签      4
...                    ...
番禺科技园   番禺科技园    应结未结    2
总计                     199

[45 rows x 1 columns]

## 分析增量数据


In [15]:
analyze(incremental_db)


数量
分公司     项目部      情况    
产园-杭州公司 --小计--       24
        上海森兰美奂创库 倒签   2
...                  ..
产园-武汉公司 高新网谷     倒签   1
总计                   77

[28 rows x 1 columns]

## 导出下发数据


In [16]:
output_dir_name = 'output'
out_dir = Path.cwd() / output_dir_name
if not out_dir.exists():
    out_dir.mkdir()

out_filename_prefix = '2022-02-18'
out_filename = f'{out_filename_prefix}-租赁平台-合同规范性检查（下发）.xlsx'

out_path = out_dir / out_filename

with pd.ExcelWriter(out_path) as writer:
    analyze(tp_db).to_excel(writer, sheet_name='本期统计')
    analyze(incremental_db).to_excel(writer, sheet_name='增量统计')
    tp_db.to_excel(writer, sheet_name='本期数据')
    incremental_db.to_excel(writer, sheet_name='增量数据')
